## Tests

Let's test the BackgroundTaskManager with various scenarios including success, failure, and retry logic.

In [ ]:
#| hide

# Setup test database using tenant infrastructure (shared with other test suites)
import os
import json
from sqlalchemy import text
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables - use absolute path
notebook_dir = Path().absolute()
if notebook_dir.name == 'nbs':
    env_path = notebook_dir / '.env'
else:
    env_path = notebook_dir / 'nbs' / '.env'

if env_path.exists():
    load_dotenv(env_path)

from fh_saas.utils_bgtsk import BackgroundTaskManager, AppJob
from fh_saas.db_tenant import get_or_create_tenant_db
from fh_saas.db_host import timestamp, gen_id

In [ ]:
#| hide

# FIXED: Added load_dotenv to resolve DB_PASS error
# Reuse the same test tenant as sql_utils (different tables, no conflicts)
print("🔧 Setting up test database...")
test_db = get_or_create_tenant_db("test_utils", "Utilities Test Tenant")

# Clean up any existing test data to make tests idempotent
test_db.conn.execute(text("DELETE FROM app_jobs"))
test_db.conn.commit()
print("   🧹 Cleaned up existing test data")

print(f"✅ Test database ready")
manager = BackgroundTaskManager(test_db)
print(f"✓ BackgroundTaskManager initialized")

🔧 Setting up test database...
ℹ️  Tenant exists: Utilities Test Tenant
   🧹 Cleaned up existing test data
✅ Test database ready
✓ BackgroundTaskManager initialized


### Test 1: Successful Task Execution

In [ ]:
#| hide

# Define a simple task function
def sync_transactions(user_id: str, count: int):
    """Simulate syncing transactions"""
    return {"user_id": user_id, "synced": count, "status": "success"}

# Submit the task
job_id, bg_task = manager.submit(
    job_type="transaction_sync",
    task_func=sync_transactions,
    user_id="user123",
    count=100
)

print(f"✓ Job submitted: {job_id}")

# Check initial status
job = manager.get_job(job_id)
assert job.status == 'pending', f"Expected 'pending', got '{job.status}'"
assert job.job_type == 'transaction_sync'
assert json.loads(job.payload) == {"user_id": "user123", "count": 100}
print(f"✓ Job status: {job.status}")

# Execute the background task synchronously for testing
bg_task.func(*bg_task.args, **bg_task.kwargs)

# Check final status
job = manager.get_job(job_id)
assert job.status == 'completed', f"Expected 'completed', got '{job.status}'"
assert job.result is not None
result = json.loads(job.result)
assert result["synced"] == 100
assert result["user_id"] == "user123"
print(f"✓ Job completed successfully")
print(f"✓ Result: {result}")

✓ Job submitted: 7af01f9ccdb34e7bbddf04a168be83e5
✓ Job status: pending
✓ Job completed successfully
✓ Result: {'user_id': 'user123', 'synced': 100, 'status': 'success'}


### Test 2: Task with Failure and Retry Logic

In [ ]:
#| hide

# Define a task that fails
def failing_task(should_fail: bool = True):
    """Simulate a task that fails"""
    if should_fail:
        raise ValueError("Simulated task failure")
    return {"status": "success"}

# Submit the failing task
job_id, bg_task = manager.submit(
    job_type="email_send",
    task_func=failing_task,
    max_retries=3,
    should_fail=True
)

print(f"✓ Failing job submitted: {job_id}")

# Execute the background task
bg_task.func(*bg_task.args, **bg_task.kwargs)

# Check status after first failure
job = manager.get_job(job_id)
print(f"✓ Job status after failure: {job.status}")
print(f"✓ Retry count: {job.retry_count}")
assert job.retry_count == 1, f"Expected retry_count=1, got {job.retry_count}"
assert job.error_log is not None
assert "ValueError: Simulated task failure" in job.error_log
print(f"✓ Error logged: {job.error_log[:100]}...")

# Simulate multiple retries to reach max_retries
for i in range(2, 4):  # Retry 2 and 3
    bg_task.func(*bg_task.args, **bg_task.kwargs)
    job = manager.get_job(job_id)
    print(f"✓ After retry {i}: status={job.status}, retry_count={job.retry_count}")

# Final check - should be failed after max retries
job = manager.get_job(job_id)
assert job.status == 'failed', f"Expected 'failed', got '{job.status}'"
assert job.retry_count == 3
assert job.completed_at is not None
print(f"✓ Job marked as failed after max retries")

✓ Failing job submitted: 3937af37fb604dfb8f0af44498c9759a
✓ Job status after failure: pending
✓ Retry count: 1
✓ Error logged: ValueError: Simulated task failure
Traceback (most recent call last):
  File "C:\Users\abhis\Documen...
✓ After retry 2: status=pending, retry_count=2
✓ After retry 3: status=failed, retry_count=3
✓ Job marked as failed after max retries


### Test 3: List and Filter Jobs

In [ ]:
#| hide

# Submit a few more jobs for testing list functionality
def dummy_task(task_id: int):
    return {"task_id": task_id}

# Submit multiple jobs
job_ids = []
for i in range(3):
    job_id, bg_task = manager.submit(
        job_type="batch_process",
        task_func=dummy_task,
        task_id=i
    )
    job_ids.append(job_id)
    bg_task.func(*bg_task.args, **bg_task.kwargs)

print(f"✓ Submitted {len(job_ids)} batch jobs")

# List all jobs
all_jobs = manager.list_jobs()
print(f"✓ Total jobs in database: {len(all_jobs)}")
assert len(all_jobs) >= 5  # 1 successful + 1 failed + 3 batch

# Filter by job type
sync_jobs = manager.list_jobs(job_type="transaction_sync")
assert len(sync_jobs) == 1
print(f"✓ Transaction sync jobs: {len(sync_jobs)}")

batch_jobs = manager.list_jobs(job_type="batch_process")
assert len(batch_jobs) == 3
print(f"✓ Batch process jobs: {len(batch_jobs)}")

# Filter by status
completed_jobs = manager.list_jobs(status="completed")
print(f"✓ Completed jobs: {len(completed_jobs)}")
assert len(completed_jobs) >= 4  # 1 sync + 3 batch

failed_jobs = manager.list_jobs(status="failed")
assert len(failed_jobs) == 1
print(f"✓ Failed jobs: {len(failed_jobs)}")

# Combined filter
batch_completed = manager.list_jobs(job_type="batch_process", status="completed")
assert len(batch_completed) == 3
print(f"✓ Batch completed jobs: {len(batch_completed)}")

✓ Submitted 3 batch jobs
✓ Total jobs in database: 5
✓ Transaction sync jobs: 1
✓ Batch process jobs: 3
✓ Completed jobs: 4
✓ Failed jobs: 1
✓ Batch completed jobs: 3


### Test 4: Task with No Return Value

In [ ]:
#| hide

# Define a task that doesn't return anything
def send_email(to: str, subject: str):
    """Simulate sending an email (no return value)"""
    # In real code, this would send an actual email
    pass

# Submit the task
job_id, bg_task = manager.submit(
    job_type="email_send",
    task_func=send_email,
    to="user@example.com",
    subject="Welcome to FHSaaS"
)

print(f"✓ Email job submitted: {job_id}")

# Execute
bg_task.func(*bg_task.args, **bg_task.kwargs)

# Check status
job = manager.get_job(job_id)
assert job.status == 'completed'
assert job.result is None  # No return value
print(f"✓ Email job completed with no return value")

✓ Email job submitted: d09666e4a2564c20b371e9037f2b7595
✓ Email job completed with no return value


### Test Summary

In [ ]:
#| hide

# Final summary
all_jobs = manager.list_jobs(limit=1000)
status_counts = {}
for job in all_jobs:
    status_counts[job.status] = status_counts.get(job.status, 0) + 1

print("\n" + "="*50)
print("TEST SUMMARY")
print("="*50)
print(f"Total jobs: {len(all_jobs)}")
for status, count in sorted(status_counts.items()):
    print(f"  {status}: {count}")
print("\n✓ All tests passed!")
print("="*50)


TEST SUMMARY
Total jobs: 6
  completed: 5
  failed: 1

✓ All tests passed!
